In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
model = tf.keras.Sequential([
   keras.layers.Dense(units=4, input_shape=[1]),
   keras.layers.Dense(units=8),
   keras.layers.Dense(units=1),
   ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 8         
                                                                 
 dense_1 (Dense)             (None, 8)                 40        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [2]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [4]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 13.6312
Epoch 2/100
1/1 [==============================] - 0s 15ms/step - loss: 5.5411
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 1.9543
Epoch 4/100
1/1 [==============================] - 0s 9ms/step - loss: 0.9562
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7797
Epoch 6/100
1/1 [==============================] - 0s 7ms/step - loss: 0.6806
Epoch 7/100
1/1 [==============================] - 0s 21ms/step - loss: 0.5934
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5162
Epoch 9/100
1/1 [==============================] - 0s 18ms/step - loss: 0.4478
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.3876
Epoch 11/100
1/1 [==============================] - 0s 16ms/step - loss: 0.3346
Epoch 12/100
1/1 [==============================] - 0s 14ms/step - loss: 0.2882
Epoch 13/100
1/1 [==============================] - 0s

In [3]:
print(model.predict([10.0]))

1/1 [==============================] - 0s 346ms/step
[[2.3182743]]


In [4]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [5]:
!pip install fastapi nest-asyncio pyngrok uvicorn

  Using cached fastapi-0.95.2-py3-none-any.whl (56 kB)
     ------------------------------------- 681.2/681.2 kB 25.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
     ---------------------------------------- 2.1/2.1 MB 38.1 kB/s eta 0:00:00
  Using cached starlette-0.27.0-py3-none-any.whl (66 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19868 sha256=f76c0f17fa1a28653c8d0a9474931689f4b4559f5e3163a3fc0173eb22a7d71b
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\5c\42\78\0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [10]:
# from fastapi import FastAPI
# from uvicorn import Config, Server
# import asyncio
# app = FastAPI()

# @app.get("/")
# async def root():
#     return {"message": "Hello World"}

# config = Config(app)
# server = Server(config=config)

# loop = asyncio.get_event_loop()
# loop.create_task(server.serve())

In [ ]:
import tensorflow as tf
import asyncio
from fastapi import FastAPI
from uvicorn import Config, Server
from pydantic import BaseModel

import numpy as np

MODEL = tf.keras.models.load_model('model/')
app = FastAPI()

class UserInput(BaseModel):
    user_input: float

@app.get('/')
async def index():
    return {"Message": "This is Index"}
config = Config(app)
server = Server(config=config)

loop = asyncio.get_event_loop()
loop.create_task(server.serve())

@app.post('/predict/')
async def predict(UserInput: UserInput):

    prediction = MODEL.predict([UserInput.user_input])

    return {"prediction": float(prediction)}

INFO:     Started server process [6816]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:63512 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:63515 - "POST /predict/ HTTP/1.1" 422 Unprocessable Entity
1/1 [==============================] - 0s 308ms/step
INFO:     127.0.0.1:63526 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:63615 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:63616 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:63616 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 [==============================] - 0s 48ms/step
INFO:     127.0.0.1:63621 - "POST /predict/ HTTP/1.1" 200 OK
